In [1]:
import matplotlib.pyplot as plt
import numpy as np
import egg.core as core
import pandas as pd

from utils.analysis_tools_lazimpa import *
from utils.analysis_from_interaction import *
from archs import Sender, Receiver
from train import loss
from models_lazimpa import LazImpaSenderReceiverRnnGS
from dataset import DataSet
import torch.nn.functional as F

In [2]:
dataset = '(3,4)'
values = 4
attributes = 3
n_epochs = 300
path = 'results/' + dataset + '_game_size_10_vsf_3'
context_unaware = False # whether original or context_unaware simulations are evaluated
lazy = True # whether the lazy agent is used
impatient = True
setting = ""
if not lazy:
    start = 'impatience_' if impatient else ''
    if context_unaware:
        setting = start + 'context_unaware'
    else:
        setting = start + 'context_aware'
        if impatient == False:
            setting = 'standard'
else:
    start = 'lazimpa' if impatient else 'lazy'
    if context_unaware:
        setting = start + '_context_unaware'
    else:
        setting = start + '_context_aware'
setting
run = 0
setting

'lazimpa_context_aware'

In [3]:
# Start what kind of variables do I need for attribute visualization

interaction = load_interaction(path,setting,run,n_epochs)
concepts, context_condition = retrieve_concepts_context(interaction,values)
messages = retrieve_messages(interaction,False,remove_after_eos=True)
message_length = MessageLengthHierarchical.compute_message_length(messages)
unique_messages, frequencies = torch.unique(messages,dim=0,return_counts=True)
unique_length = MessageLengthHierarchical.compute_message_length(unique_messages)

m = [f"{ *m, }" for m in messages.numpy()] # as str for hashing
l = message_length.numpy()
cx = context_condition
fint = [list(map(int,list(f))) for c,f in concepts]
fix = [f"{v}" for v in fint] # fix as string
c = [list( map(int, list( [a if fa == 1. else -1 for a,fa in zip(c[0],f) ] ))) for c,f in concepts]
um = [f"{ *m, }" for m in unique_messages.numpy()]
ul = unique_length.numpy()
f = frequencies.numpy()

results/(3,4)_game_size_10_vsf_3/lazimpa_context_aware/0/interactions/train/epoch_300/interaction_gpu0
torch.Size([1784, 20, 12])


In [4]:
# make one DataFrame containing all the data we have
in_list = []
for i in range(message_length.shape[0]):
    in_list.append((fix[i],fint[i], np.where(concepts[i][1]==1.0)[0].tolist(), sum(fint[i]), c[i], cx[i], m[i], l[i])) # np.where(cc[i][1]==1.0)[0].tolist()[0]
df = pd.DataFrame(in_list,columns=["fixed", "fixed int","which fixed indices", "nr attributes", "concept", "context condition", "message", "message length"])
df

,fixed,fixed int,which fixed indices,nr attributes,concept,context condition,message,message length
0,"[1, 1, 0]","[1, 1, 0]","[0, 1]",2,"[0, 3, -1]",0,"(0, 0, 0, 0)",0
1,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[2, 0, 1]",2,"(8, 0, 0, 0)",1
2,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[3, 2, 1]",2,"(15, 0, 0, 0)",1
3,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[0, 3, 1]",0,"(0, 0, 0, 0)",0
4,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[0, 2, 2]",1,"(10, 0, 0, 0)",1
...,...,...,...,...,...,...,...,...
1779,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[3, 1, 3]",2,"(5, 0, 0, 0)",1
1780,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[1, 0, 3]",0,"(9, 0, 0, 0)",1
1781,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[0, 3, 2]",1,"(0, 0, 0, 0)",0
1782,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[3, 2, 1]",1,"(8, 0, 0, 0)",1


In [5]:
# calculate additional data that can be merged
unique_df = pd.DataFrame([(m,f) for m,f in zip(list(um),list(f))],columns=["message", "usage frequency"])
total_data_df = df.merge(unique_df,how='right',on="message")
total_data_df

,fixed,fixed int,which fixed indices,nr attributes,concept,context condition,message,message length,usage frequency
0,"[1, 1, 0]","[1, 1, 0]","[0, 1]",2,"[0, 3, -1]",0,"(0, 0, 0, 0)",0,226
1,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[0, 3, 1]",0,"(0, 0, 0, 0)",0,226
2,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[0, 1, 1]",1,"(0, 0, 0, 0)",0,226
3,"[1, 1, 0]","[1, 1, 0]","[0, 1]",2,"[0, 3, -1]",0,"(0, 0, 0, 0)",0,226
4,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[0, 0, 3]",0,"(0, 0, 0, 0)",0,226
...,...,...,...,...,...,...,...,...,...
1779,"[0, 1, 0]","[0, 1, 0]",[1],1,"[-1, 2, -1]",0,"(15, 15, 0, 0)",2,6
1780,"[0, 1, 0]","[0, 1, 0]",[1],1,"[-1, 2, -1]",0,"(15, 15, 0, 0)",2,6
1781,"[0, 1, 0]","[0, 1, 0]",[1],1,"[-1, 2, -1]",0,"(15, 15, 0, 0)",2,6
1782,"[0, 1, 0]","[0, 1, 0]",[1],1,"[-1, 2, -1]",0,"(15, 15, 0, 0)",2,6


In [6]:
def many_hot(input,data):
    output = torch.zeros([sum(data.properties_dim)])
    start = 0

    for elem, dim in zip(input, data.properties_dim):
        output[start + elem] = 1
        start += dim

    return output

In [7]:
def get_sender_input(data, concept_idx, context_condition, encoding_func):
        sender_concept, sender_context = data.get_sample(concept_idx, context_condition)

        # subset such that only target objects are presented to sender and receiver
        sender_targets = sender_concept[0]
        sender_input = [obj for obj in sender_targets]

        # append context objects
        # get context of relevant context condition
        for distractor_objects, context_cond in sender_context:
            if context_cond == context_condition:
                # add distractor objects for the sender
                for obj in distractor_objects:
                    sender_input.append(obj)

        # ENCODE and return as TENSOR
        sender_input = torch.stack([many_hot(elem,data) for elem in sender_input])
        return sender_input

In [8]:
def get_receiver_input(data, concept_idx, context_condition, encoding_func):
        receiver_concept, receiver_context = data.get_sample(concept_idx, context_condition)

        # subset such that only target objects are presented to sender and receiver
        receiver_targets = receiver_concept[0]
        receiver_input = [obj for obj in receiver_targets]

        for distractor_objects, context_cond in receiver_context:
            if context_cond == context_condition:
                # add distractor objects for the receiver
                for obj in distractor_objects:
                    receiver_input.append(obj)
        
        random.shuffle(receiver_input)
        receiver_label = [idx for idx, obj in enumerate(receiver_input) if obj in receiver_targets]
        receiver_label = torch.Tensor(receiver_label).to(torch.int64)
        receiver_label = F.one_hot(receiver_label, num_classes=data.game_size * 2).sum(dim=0).float()

        # ENCODE and return as TENSOR
        receiver_input = torch.stack([many_hot(elem,data) for elem in receiver_input])
        return receiver_label, receiver_input

In [9]:
# calculate meanings (concept x message for fine context)
runs = 7
data = DataSet(list(itertools.repeat(values, attributes)),
    game_size=10,
    scaling_factor=10,
    zero_shot=True,
    zero_shot_test=False,
    sample_context=False)

concept_fixed_set = data.get_all_concepts()
set_concepts = [list( map(int, list( [a if fa == 1. else -1 for a,fa in zip(c[0],f) ] ))) for c,f in concept_fixed_set]
fint_set = [list(map(int,list(f))) for c,f in concept_fixed_set]
sender_input = torch.cat([get_sender_input(data, i, sum(f)-1, data._many_hot_encoding).unsqueeze(0) for i,f in zip(range(len(concept_fixed_set)),fint_set)],dim=0)
listener,_,model = load_listener(path, setting, run, attributes, values, context_unaware)
sender = model.sender 
messages_original = [sender(sender_input) for r in range(runs)]
messages_tensor = torch.cat([messages_without_after_eos(msg,0.0).argmax(dim=-1).unsqueeze(1) for msg in messages_original],dim=1)

Creating train_ds, val_ds and test_ds...


100%|██████████| 124/124 [00:00<00:00, 123861.32it/s]

Length of train and validation datasets: 0 / 0
Length of test dataset: 0


In [10]:
# get whether the listener can understand the messages
receiver_input_list = []
labels_list = []
for i,f in zip(range(len(concept_fixed_set)),fint_set):
    labels, receiver_input = get_receiver_input(data, i, sum(f)-1, data._many_hot_encoding)
    receiver_input_list.append(receiver_input.unsqueeze(0))
    labels_list.append(labels.unsqueeze(0).unsqueeze(0).unsqueeze(0))

receiver_input = torch.cat(receiver_input_list,dim=0)
labels = torch.cat(labels_list,dim=0).repeat(1,runs,attributes + 1, 1)

eos_mask_total = 0.0 != messages_tensor # remember this is fore several runs
# only values before eos, to switch
eos_excluded_cumulative = torch.cat([ eos_mask_total[:,:,:i+1].sum(dim=2).unsqueeze(-1) == i+1 for i in range(attributes +1) ],dim=-1)
# until eos, evaluating changes
eos_included_cumulative = torch.cat([torch.ones_like(eos_excluded_cumulative[:,:,-1]).unsqueeze(-1).bool(), eos_excluded_cumulative[:,:,:-1]],dim=-1)
eos_exact = (eos_included_cumulative != eos_mask_total) # only symbol first eos

predictions = torch.cat([(listener(msg,receiver_input) > 0).float().unsqueeze(1) for msg in messages_original],dim=1)
# remove after eos prediction
predictions = torch.where(eos_exact.unsqueeze(-1),predictions,labels)
score = (predictions != labels)

In [11]:
def aggregate_messages(messages):
    message_counts = messages.value_counts().to_dict()
    unique_messages = list(message_counts.keys())
    frequencies = sum(list(message_counts.values()))
    return unique_messages, frequencies

In [12]:
# make one DataFrame containing all the concepts and messages used for them in fine context

# make data hashable, and calc averages etc. 
messages_set = [list(set([f"{ *mes, }" for mes in msg_5])) for msg_5 in [msg.tolist() for msg in messages_tensor]]
messages_str = [[f"{ *mes, }" for mes in msg_5] for msg_5 in [msg.tolist() for msg in messages_tensor]]
concepts_str = [f"{ *con, }" for con in set_concepts]
wrong_in_total = score.sum(dim=(2,3)).bool().sum(dim=-1)/runs
wrong_per_message = score.sum(dim=(2,3))/20

# create DataFrame to calc average wrongness per message etc
in_list = []
for i in range(len(messages_str)):
    for j,m in enumerate(messages_str[i]):
        in_list.append((m,concepts_str[i],wrong_per_message[i][j].item()))
meaning_message_df = pd.DataFrame(in_list,columns=["message","meaning","wrong %"])
meaning_message_df = meaning_message_df.groupby(['meaning','message'])['wrong %'].apply(lambda l: l.mean()).reset_index().round(2)
with_messages = meaning_message_df.groupby(['meaning'])['message'].apply(lambda m:list(m.value_counts().to_dict().keys())).reset_index()
with_wrong = meaning_message_df.groupby(['meaning'])['wrong %'].apply(lambda m:list(m.value_counts().to_dict().keys())).reset_index()
wrong_per_message_df = with_messages.merge(with_wrong,how='right',on="meaning")

# create DataFrame with all data
in_list = []
for i in range(len(messages_set)):
    in_list.append((concepts_str[i],fint_set[i],wrong_in_total[i].item()))
meaning_df = pd.DataFrame(in_list,columns=["meaning","fixed meaning int","not all correct percent total"]).round(1)
#meaning_df.round(1).drop(meaning_df[meaning_df["not all correct percent total"] == 0.0].index)
meaning_df = meaning_df.merge(wrong_per_message_df,how='right',on='meaning')
meaning_df

,meaning,fixed meaning int,not all correct percent total,message,wrong %
0,"(-1, -1, 0)","[0, 0, 1]",1.0,"[(13, 0, 0, 0), (6, 0, 0, 0)]","[0.4, 0.28]"
1,"(-1, -1, 1)","[0, 0, 1]",0.0,"[(8, 0, 0, 0)]",[0.0]
2,"(-1, -1, 2)","[0, 0, 1]",0.4,"[(11, 0, 0, 0)]",[0.02]
3,"(-1, -1, 3)","[0, 0, 1]",1.0,"[(9, 0, 0, 0)]",[0.17]
4,"(-1, 0, -1)","[0, 1, 0]",1.0,"[(4, 0, 0, 0)]",[0.4]
...,...,...,...,...,...
119,"(3, 3, -1)","[1, 1, 0]",0.7,"[(3, 7, 0, 0)]",[0.11]
120,"(3, 3, 0)","[1, 1, 1]",0.0,"[(13, 0, 0, 0), (3, 0, 0, 0), (3, 13, 0, 0)]",[0.0]
121,"(3, 3, 1)","[1, 1, 1]",0.9,"[(5, 0, 0, 0), (8, 0, 0, 0)]","[0.05, 0.03]"
122,"(3, 3, 2)","[1, 1, 1]",0.0,"[(11, 0, 0, 0)]",[0.0]


In [13]:
# create a similar dataframe but sorted by message
with_wrong = meaning_message_df.groupby(['message'])['wrong %'].apply(lambda m:list(m.value_counts().to_dict().keys())).reset_index()
with_meaning = meaning_message_df.groupby(['message'])['meaning'].apply(aggregate_messages).reset_index()
with_meaning[['unique meanings', 'frequency used by meaning']] = pd.DataFrame(with_meaning['meaning'].tolist(), index=with_meaning.index)
message_df = with_meaning.merge(with_wrong,how='right',on="message").drop("meaning",axis=1)
message_df # TODO true meaning with wrong == 0

,message,unique meanings,frequency used by meaning,wrong %
0,"(0, 0, 0, 0)","[(0, -1, -1), (0, -1, 0), (0, 3, 2), (0, 3, 1)...",20,"[0.0, 0.1, 0.05, 0.02, 0.07, 0.12, 0.22, 0.16,..."
1,"(1, 0, 0, 0)","[(2, 1, -1), (2, 1, 0), (2, 1, 1), (2, 1, 2), ...",10,"[0.0, 0.05, 0.24, 0.31, 0.1, 0.07]"
2,"(1, 8, 0, 0)","[(2, 3, 1)]",1,[0.0]
3,"(10, 0, 0, 0)","[(-1, 2, -1), (-1, 2, 2), (0, 2, 2), (1, 2, 2)...",7,"[0.0, 0.4, 0.05, 0.25]"
4,"(10, 14, 0, 0)","[(1, 2, 2)]",1,[0.0]
5,"(10, 7, 0, 0)","[(3, 2, 2)]",1,[0.0]
6,"(11, 0, 0, 0)","[(-1, -1, 2), (-1, 0, 2), (-1, 1, 2), (-1, 3, ...",16,"[0.0, 0.05, 0.02, 0.08, 0.2, 0.1, 0.55, 0.4, 0..."
7,"(11, 1, 0, 0)","[(2, 1, 2)]",1,[0.05]
8,"(11, 12, 0, 0)","[(2, 0, 2)]",1,[0.0]
9,"(11, 14, 0, 0)","[(1, -1, 2)]",1,[0.0]


In [ ]:
# find true meaning


# Data Analysis

In [6]:
message_length.float().mean() # weighted by occurence

tensor(1.5368)

In [7]:
# How long are the messages on average used for a certain fix attribute number
scores = message_length_per_hierarchy_level(interaction, attributes)
scores # These values are correct!

message length tensor([1, 1, 1,  ..., 1, 1, 2])
hierarchical [1.607, 1.544, 1.53]


[1.607, 1.544, 1.53]

In [8]:
# How long are the messages on average used in the different context conditions
grouped_by_context = df.groupby(["context condition"])['message length'].apply(lambda l: l.mean()).reset_index()
grouped_by_context

,context condition,message length
0,0,1.281333
1,1,1.601757
2,2,1.930481


In [9]:
# how long are the messages on average for different numbers of fixed attributes
grouped_by_nr_fixed_attributes = df.groupby(["nr attributes","context condition"])["message length"].apply(lambda l: l.mean()).reset_index()
grouped_by_nr_fixed_attributes

,nr attributes,context condition,message length
0,1,0,1.607143
1,2,0,1.363036
2,2,1,1.729730
3,3,0,1.171355
4,3,1,1.503876
5,3,2,1.930481


In [10]:
# how often are messages used with different lengths in total
#grouped_by_length = pd.DataFrame([(l,f) for l,f in zip(ul,f)],columns=["length","frequency"])
grouped_by_length = total_data_df.groupby(["message length"])['usage frequency'].apply(lambda f: (f.mean(),f.std(ddof=False))).reset_index()
grouped_by_length[['mean usage frequency', 'std usage frequency']] = pd.DataFrame(grouped_by_length['usage frequency'].tolist(), index=grouped_by_length.index)
grouped_by_length = grouped_by_length.drop('usage frequency',axis=1)
grouped_by_length

,message length,mean usage frequency,std usage frequency
0,1,71.648045,26.524830
1,2,17.269321,11.625401
2,3,6.034483,2.994049


In [406]:
# how long are messages used for a certain number and position of fixed attributes (only in fine context) # used means all the messages that are used also if they are the same used twice
context = 1 # 1 means 2 (fine), 2 means 1, 0 means -3 (coarse) für 3 fixed attributes
in_list = []
for i in range(message_length.shape[0]):
    #if cx[i].sum() == 1: # all that have exactly 1 fixed attribute
    if cx[i] + context == sum(fint[i]): # Only the ones with specific context
        content = [int(sl) if sl == 1. else "" for sl in list(fint[i])]
        content.append(l[i])
        in_list.append(content) # np.where(cc[i][1]==1.0)[0].tolist()[0]
df = pd.DataFrame(in_list,columns=["fixed 1", "fixed 2", "fixed 3", "av message length"])
df = df.groupby(['fixed 1','fixed 2','fixed 3'])['av message length'].apply(lambda l: l.mean()).reset_index()
df

TypeError: 'RnnReceiverGS' object is not subscriptable